<a href="https://colab.research.google.com/github/Dmitriy6655/Sem_9_Ansambli/blob/main/Sem_9_Ansambli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ДЗ_09**
Урок 9. Ансамбли на практике

Ваша задача — реализовать алгоритм случайного леса для задачи классификации и применить его к набору данных.

Шаги выполнения задания:

    Импортируйте необходимые библиотеки: numpy, pandas, sklearn.

    Загрузите набор данных для задачи классификации. Вы можете использовать любой доступный набор данных или выбрать один из популярных, таких как Iris, Wine или MNIST.

    Проведите предварительную обработку данных, включая масштабирование и разделение на обучающую и тестовую выборки.

    Реализуйте алгоритм случайного леса. Включите в него функции для построения деревьев решений, выбора случайных признаков и голосования для принятия решений.

    Обучите вашу модель случайного леса на обучающей выборке.

    Оцените производительность модели на тестовой выборке, используя метрики классификации, такие как точность, полнота и F1-мера.

    Проведите сравнение результатов вашей модели со стандартной реализацией случайного леса из библиотеки scikit-learn.



# ШАГ 1

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



# ШАГ 2

In [3]:
# Загружаем набор данных Iris
iris = load_iris()

# ШАГ 3

In [4]:
# Преобразуем данные в DataFrame
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

df.head()



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
# Проверим наличие пропущенных значений


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [19]:
# Создаём объект StandardScaler для масштабирования данных
scaler = StandardScaler()

# Применяем масштабирование
X_scaled = scaler.fit_transform(X)

# Преобразуем результат обратно в DataFrame для наглядности
X_scaled = pd.DataFrame(X_scaled, columns=iris.feature_names)

print(X_scaled.head())

X = iris.data
y = iris.target


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0          -0.900681          1.019004          -1.340227         -1.315444
1          -1.143017         -0.131979          -1.340227         -1.315444
2          -1.385353          0.328414          -1.397064         -1.315444
3          -1.506521          0.098217          -1.283389         -1.315444
4          -1.021849          1.249201          -1.340227         -1.315444


In [28]:
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ШАГ 4

In [20]:
# Функция для построения дерева решений
def build_simple_tree(X, y, max_depth=3, depth=0):
    # Условие остановки: достигнута максимальная глубина или все метки одинаковы
    if len(np.unique(y)) == 1 or depth == max_depth:
        # Возвращаем наиболее частую метку
        return np.bincount(y).argmax()

    # Выбираем случайный признак
    feature = np.random.randint(X.shape[1])
    # Выбираем порог для разделения как среднее значение признака
    threshold = np.mean(X[:, feature])

    # Делим данные на две группы
    left_mask = X[:, feature] <= threshold
    right_mask = ~left_mask

    if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
        # Если одна из групп пуста, возвращаем наиболее частую метку
        return np.bincount(y).argmax()

    # Рекурсивно строим дерево для обеих частей
    left_tree = build_simple_tree(X[left_mask], y[left_mask], max_depth, depth + 1)
    right_tree = build_simple_tree(X[right_mask], y[right_mask], max_depth, depth + 1)

    # Возвращаем структуру дерева
    return (feature, threshold, left_tree, right_tree)

In [26]:
# Прогнозирование
def predict_tree(tree, X_sample):
    if isinstance(tree, (int, np.integer)):
        return tree

    feature, threshold, left_tree, right_tree = tree

    if X_sample[feature] <= threshold:
        return predict_tree(left_tree, X_sample)
    else:
        return predict_tree(right_tree, X_sample)

# Построение случайного леса
def build_random_forest(X, y, n_trees=5, max_depth=3):
    trees = []
    for _ in range(n_trees):
        # Bootstrap выборка
        indices = np.random.choice(len(X), len(X), replace=True)
        X_sample = X[indices]
        y_sample = y[indices]

        # Построение дерева
        tree = build_simple_tree(X_sample, y_sample, max_depth=max_depth)
        trees.append(tree)

    return trees

# Прогнозирование с помощью случайного леса
def predict_random_forest(trees, X):
    predictions = []
    for tree in trees:
        preds = np.array([predict_tree(tree, x) for x in X])
        predictions.append(preds)

    # Мажоритарное голосование
    predictions = np.array(predictions)
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

    return final_predictions





# ШАГ5

In [29]:
trees = build_random_forest(X_train, y_train, n_trees=3, max_depth=3)

# Прогнозирование на тестовых данных
y_pred = predict_random_forest(trees, X_test)


# ШАГ 6

In [30]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Моя реализация - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')


Моя реализация - Accuracy: 0.8667, Precision: 0.8974, Recall: 0.8788, F1: 0.8653


# ШАГ 7

In [32]:
from sklearn.ensemble import RandomForestClassifier

# Обучаем RandomForestClassifier
rf_sklearn = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)
rf_sklearn.fit(X_train, y_train)

# Прогнозы на тестовой выборке
y_pred_sklearn = rf_sklearn.predict(X_test)

# Оценка метрик для scikit-learn модели
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
precision_sklearn = precision_score(y_test, y_pred_sklearn, average='macro')
recall_sklearn = recall_score(y_test, y_pred_sklearn, average='macro')
f1_sklearn = f1_score(y_test, y_pred_sklearn, average='macro')

print(f'Реализация Sklearn - Accuracy: {accuracy_sklearn:.4f}, Precision: {precision_sklearn:.4f}, Recall: {recall_sklearn:.4f}, F1: {f1_sklearn:.4f}')


Реализация Sklearn - Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000


**Выводы:** Моя реализация алгоритма случайного леса показала хорошие результаты. Но из-за более сложных и оптимизированных методов для построения деревьев и выбора признаков, Sklearn показал максимальный результат.